In [16]:
import pandas as pd

In [202]:
columns = [52, 26, 10, 205, 186, 18, 48, 60, 198, 233, 160, 142, 227, 114, 22, 50, 133, 24]
columns = [str(i) for i in columns]

In [203]:
dfs = {}

for i in range(2012, 2019):
    dfs[i] = pd.read_csv("rosstat_{0}_without_prev.csv".format(i))

status_df = pd.read_csv('status1.csv')

In [204]:
def check_inn(year, inn): # checks if inn exists in the given year
    if year > 2018 or year < 2012:
        return False
    df = dfs[year]
    return not df[df['5'] == inn].empty


def if_exists_later(year, inn):
    if year > 2018:
        return True
    for i in range(year, 2019):
        if check_inn(i, inn):
            return True
    return False
    

def get_columns(year, inn):
    df = dfs[year]
    return df[df['5'] == inn][['5'] + columns] # save INN to merge on that


def take_three_prev(year):
    print(year, "is started")
    if year < 2014:
        return pd.DataFrame()
    
    current_df = dfs[year]
    inns = current_df['5'].values
    df_final = pd.DataFrame()
    ii = 0
    for elem in inns:
        ii += 1
        if check_inn(year-2, elem) and check_inn(year-1, elem):
            # exists three years in a row
            df1 = get_columns(year, elem).add_suffix('_1')
            df2 = get_columns(year-1, elem).add_suffix('_2')
            df3 = get_columns(year-2, elem).add_suffix('_3')

            df_out = pd.merge(df1, df2, left_on='5_1', right_on='5_2', how='left')
            del df_out['5_2']
            df_out = pd.merge(df_out, df3, left_on='5_1', right_on='5_3', how='left')
            del df_out['5_3']
            df_out['curr_year'] = year
            if not if_exists_later(year+1, elem) and status_df[status_df['INN'] == elem]['STATUS'].values[0] == 1:
                df_out['is_closed'] = 1
            else:
                df_out['is_closed'] = 0
            
            df_final = df_final.append(df_out)
            
        if ii % 2000 == 0:
            print(int(ii / current_df.shape[0] * 100), '%', year)
    
    print(year, "is done")
    return df_final
       
            
def get_3year_lag():
    df_res = pd.DataFrame()
    for year in range(2012, 2019):
        df_res = df_res.append(take_three_prev(year))

    df_res.rename(columns={"5_1": "INN"}, inplace=True)
    df_res.reset_index(drop=True, inplace=True)
    df_res.to_csv("3year_lag.csv", index=False)
    return df_res

In [205]:
def take_two_prev(year):
    print(year, "is started")
    if year < 2013:
        return pd.DataFrame()
    
    current_df = dfs[year]
    inns = current_df['5'].values
    df_final = pd.DataFrame()
    ii = 0
    for elem in inns:
        ii += 1
        if not check_inn(year+1, elem) and not check_inn(year-2, elem) and check_inn(year-1, elem):
            # exists only two years in a row
            df1 = get_columns(year, elem).add_suffix('_1')
            df2 = get_columns(year-1, elem).add_suffix('_2')
            df_out = pd.merge(df1, df2, left_on='5_1', right_on='5_2', how='left')
            del df_out['5_2']
            df_out['curr_year'] = year
            if not if_exists_later(year+1, elem) and status_df[status_df['INN'] == elem]['STATUS'].values[0] == 1:
                df_out['is_closed'] = 1
            else:
                df_out['is_closed'] = 0
            
            df_final = df_final.append(df_out)
            
        if ii % 2000 == 0:
            print(int(ii / current_df.shape[0] * 100), '%', year)
    
    print(year, "is done")
    return df_final


def get_2year_lag():
    df_res = pd.DataFrame()
    for year in range(2012, 2019):
        df_res = df_res.append(take_two_prev(year))

    df_res.rename(columns={"5_1": "INN"}, inplace=True)
    df_res.reset_index(drop=True, inplace=True)
    df_res.to_csv("2year_lag.csv", index=False)
    return df_res

In [206]:
def take_one_prev(year):
    print(year, "is started")
    if year < 2012:
        return pd.DataFrame()
    
    current_df = dfs[year]
    inns = current_df['5'].values
    df_final = pd.DataFrame()
    ii = 0
    for elem in inns:
        ii += 1
        if not check_inn(year+1, elem) and not check_inn(year-1, elem):
            # exists three years in a row
            df1 = get_columns(year, elem).add_suffix('_1')
            df_out = df1
            df_out['curr_year'] = year
            if not if_exists_later(year+1, elem) and status_df[status_df['INN'] == elem]['STATUS'].values[0] == 1:
                df_out['is_closed'] = 1
            else:
                df_out['is_closed'] = 0
            # if does not appear in any year to 2018
            df_final = df_final.append(df_out)
            
        if ii % 2000 == 0:
            print(int(ii / current_df.shape[0] * 100), '%', year)
    
    print(year, "is done")
    return df_final


def get_1year_lag():
    df_res = pd.DataFrame()
    for year in range(2012, 2019):
        df_res = df_res.append(take_one_prev(year))

    df_res.rename(columns={"5_1": "INN"}, inplace=True)
    df_res.reset_index(drop=True, inplace=True)
    df_res.to_csv("1year_lag.csv", index=False)
    return df_res

In [207]:
get_3year_lag()

2012 is started
2013 is started
2014 is started
24 % 2014
48 % 2014
72 % 2014
96 % 2014
2014 is done
2015 is started
18 % 2015
37 % 2015
56 % 2015
74 % 2015
93 % 2015
2015 is done
2016 is started
15 % 2016
31 % 2016
46 % 2016
62 % 2016
78 % 2016
93 % 2016
2016 is done
2017 is started
15 % 2017
30 % 2017
45 % 2017
60 % 2017
75 % 2017
90 % 2017
2017 is done
2018 is started
15 % 2018
30 % 2018
45 % 2018
61 % 2018
76 % 2018
91 % 2018
2018 is done


,INN,52_1,26_1,10_1,205_1,186_1,18_1,48_1,60_1,198_1,...,160_3,142_3,227_3,114_3,22_3,50_3,133_3,24_3,curr_year,is_closed
0,3525118128,0,3000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014,0
1,2460002949,16946000,774683000,0,6175000,0,0,143698000,1052000,16946000,...,0,0,-138063000,94000,4607000,0,0,9764000,2014,0
2,2312193320,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014,0
3,2311147627,0,640000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014,0
4,2310163947,0,6963000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20655,7901547271,0,3443000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
20656,2721224138,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
20657,7901546510,0,354000,0,0,0,0,0,0,0,...,0,0,0,0,434000,0,0,0,2018,0
20658,7901537837,0,4574000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2724000,2018,0


In [208]:
get_2year_lag()

2012 is started
2013 is started
28 % 2013
57 % 2013
85 % 2013
2013 is done
2014 is started
24 % 2014
48 % 2014
72 % 2014
96 % 2014
2014 is done
2015 is started
18 % 2015
37 % 2015
56 % 2015
74 % 2015
93 % 2015
2015 is done
2016 is started
15 % 2016
31 % 2016
46 % 2016
62 % 2016
78 % 2016
93 % 2016
2016 is done
2017 is started
15 % 2017
30 % 2017
45 % 2017
60 % 2017
75 % 2017
90 % 2017
2017 is done
2018 is started
15 % 2018
30 % 2018
45 % 2018
61 % 2018
76 % 2018
91 % 2018
2018 is done


,INN,52_1,26_1,10_1,205_1,186_1,18_1,48_1,60_1,198_1,...,160_2,142_2,227_2,114_2,22_2,50_2,133_2,24_2,curr_year,is_closed
0,4339005986,172000,1873000,0,0,0,0,0,67000,172000,...,0,0,0,0,17000,0,0,0,2013,0
1,3665034372,0,13037000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2013,0
2,4019000033,0,2073000,0,107000,0,0,0,0,0,...,0,44000,0,0,0,269000,0,0,2013,1
3,3906052638,0,14813000,0,0,0,3862000,0,0,0,...,0,0,0,0,0,0,0,0,2013,0
4,3019005657,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2013,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7331,1435224696,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
7332,2130076815,0,0,0,0,0,0,0,0,0,...,0,0,0,-95000,0,0,0,0,2018,0
7333,2013001809,0,338088000,0,0,0,0,103565000,0,0,...,0,0,0,-564000,0,149000000,0,0,2018,0
7334,1435168900,0,720000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0


In [209]:
get_1year_lag()

2012 is started
46 % 2012
93 % 2012
2012 is done
2013 is started
28 % 2013
57 % 2013
85 % 2013
2013 is done
2014 is started
24 % 2014
48 % 2014
72 % 2014
96 % 2014
2014 is done
2015 is started
18 % 2015
37 % 2015
56 % 2015
74 % 2015
93 % 2015
2015 is done
2016 is started
15 % 2016
31 % 2016
46 % 2016
62 % 2016
78 % 2016
93 % 2016
2016 is done
2017 is started
15 % 2017
30 % 2017
45 % 2017
60 % 2017
75 % 2017
90 % 2017
2017 is done
2018 is started
15 % 2018
30 % 2018
45 % 2018
61 % 2018
76 % 2018
91 % 2018
2018 is done


,INN,52_1,26_1,10_1,205_1,186_1,18_1,48_1,60_1,198_1,...,160_1,142_1,227_1,114_1,22_1,50_1,133_1,24_1,curr_year,is_closed
0,3524011573,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2012,1
1,2308125775,0,191840000,0,1236000,0,0,0,220000,0,...,0,0,100000,2498000,18400000,500000,0,0,2012,0
2,2319039045,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2012,0
3,2536183021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2012,1
4,2536184882,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2012,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13831,3818026678,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
13832,2016001719,29000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
13833,2130099072,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
13834,2130034029,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2018,0
